In [ ]:
### 加载模型：Qwen2.5-0.5B-Instruct模型
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = '/your/path/of/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.84it/s]


In [ ]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

# 模型参数设置

无论是本地模型推理，还是API调用大模型进行交互，你都需要通过配置一些参数以获得不同的提示结果。调整这些设置对于提高响应的可靠性非常重要，你可能需要进行一些实验才能找出适合您的用例的正确设置。以下是使用不同LLM提供程序时会遇到的常见设置：

- max length：max length用于指定大模型上下文文本的总长度上限，包括输入文本（prompt）和生成的新文本两部分。但是一般不用这个参数，因为限制为输入和输出总长，如果超过这个范围，会报错，而一般你并不能预测到能生成多少内容，所以一般用max new tokens比较多。
- max new tokens：指定生成文本的新生成部分的长度上限，不包括输入文本。
> 上面两个参数是通用参数，在[transformers官方代码](https://github.com/huggingface/transformers/blob/main/src/transformers/generation/configuration_utils.py#L104)上，本地推理的时候基本名称不变，但是使用API的时候由于模仿了ChatGPT的API格式，最大生成文本长度限制可能是max_tokens，因此在实际使用的时候需要注意。


In [ ]:
## Qwen推理需要max new tokens
## 会尽量在范围内回答，但是如果超出范围，会截断


### 在范围内生成
max_new_tokens=128
prompt="""请你写一段简单的描述春天、非常优美的的句子"""
print(inference(model,tokenizer,prompt,max_new_tokens=max_new_tokens))


### 生成内容太多，会截断
prompt="""请写一个科幻故事，详细描述人类首次登陆火星的全过程，从飞船脱离地球轨道开始，包括宇航员的心理活动、遇到的技术问题、火星表面的环境细节，以及登陆成功后的第一小时行动。要求内容至少 800 字，尽量详细，不要省略任何关键步骤。"""
print(inference(model,tokenizer,prompt,max_new_tokens=max_new_tokens))


### 扩大最长生成长度，生成更多内容，而且不会截断
max_new_tokens=2048
prompt="""请写一个科幻故事，详细描述人类首次登陆火星的全过程，从飞船脱离地球轨道开始，包括宇航员的心理活动、遇到的技术问题、火星表面的环境细节，以及登陆成功后的第一小时行动。要求内容至少 800 字，尽量详细，不要省略任何关键步骤。"""
print(inference(model,tokenizer,prompt,max_new_tokens=max_new_tokens))

{'response': '春日里，万物复苏，大地披上了嫩绿的新装，柔和的阳光如同细腻的丝绸轻抚着每一寸土地，空气中弥漫着清新的花香与泥土的气息，仿佛大自然最温柔的呢喃，令人心旷神怡。', 'generated_tokens_count': 55}
{'response': '【序章】\n2165年，距离地球3.5亿公里处，人类的“曙光号”探测飞船正以每秒7.7公里的速度穿越太空，向太阳系的尽头驶去。船内，五名宇航员正坐在自己的位置上，他们的脸上写满了紧张和期待。这不仅仅是一次普通的航天任务，更是人类历史上的里程碑，人类第一次真正意义上踏足火星。\n“曙光号”飞船内部的宽敞空间与舱壁的银色反射着阳光。舱内的空气循环系统运转良好，舱内温度保持在恒定范围内。', 'generated_tokens_count': 128}
{'response': '在遥远的未来，地球已经步入了科技高度发达的时代，人类对宇宙的探索热情也日益高涨。2067年，人类的航天事业取得了前所未有的成就，人类终于实现了首次登陆火星的梦想。这一天，编号为“天问”的大型运输飞船正载着人类第一批火星探测任务的成员——艾米莉亚、迈克尔、莉娜和杰克，向着火星进发。\n\n飞船“天问”在发射后，以接近光速的速度冲破大气层，向火星进发。飞船内部，四个宇航员都处在极度紧张的状态中。他们相互鼓励，共同面对即将到来的未知挑战。艾米莉亚坐在操纵台前，双手紧握，专注地看着飞船前方的显示器。她心中充满了激动与不安，因为她知道，一旦错过这个机会，可能永远无法再踏上这颗红色星球。她深知，这不仅是一次探险之旅，更是一次改变人类历史的重要旅程。\n\n与此同时，飞船遭遇了技术故障。导航系统出现了一些错误信号，导致飞船偏离了预定航线。面对这样的危机，艾米莉亚冷静地指挥着飞船调整姿态，将它重新调整到正确的轨道。她一边与地面指挥部保持联系，报告情况，一边密切关注着飞船的状态。在她的努力下，飞船最终恢复了正常运行。\n\n经过数周的飞行，飞船终于进入了火星轨道，距离火星越来越近。此时，飞船外部的摄像头捕捉到了令人震撼的画面：火星上呈现出一片红尘，仿佛是宇宙中一块巨大的火炭，而火星的天空则是无边无际的深邃夜空，星河璀璨，银河倾泻，美得令人窒息。飞船逐渐降速，进入降落阶段。艾米莉亚站在操纵台上，目不转睛地看着下方的火星地表，心中充满了期待和激

- temperature：用于控制生成文本的随机性和创造性。它影响模型在预测下一个 token 时对概率分布的处理方式。temperature范围在[0,1]之间，参数值越小，模型就会返回越确定的一个结果。
- top_p：也称为核采样，Nucleus Sampling，是一种用于控制生成文本随机性的策略参数，它与之前提到的 temperature 类似，但采用了不同的机制来筛选下一个可能的 token。
- top_k：top_k（Top-K 采样） 是一种控制生成文本随机性的策略参数，它通过限制模型在预测下一个 token 时的选择范围，来平衡生成内容的多样性和可靠性。
> 一般建议是改变temperature、top_p、top_k其中一个参数就行，不用都调整，因为都是为了控制回答的多样性。

- do_sample：用于控制生成过程中是否使用随机采样策略。它是决定 temperature、top_k、top_p 等随机性参数是否生效的开关，默认是False。



In [24]:
## 温度参数，如果越低，生成越单调，每次生成的重复性越高；如果越高，生成越多样

temperature=0.1
prompt="""周末要带孩子出去玩，推荐5种不同的活动。回答尽量简单"""
print(inference(model,tokenizer,prompt,temperature=temperature))

{'response': '当然可以，以下是五种适合周末带孩子外出的活动建议：\n\n1. **公园野餐**：选择一个风景优美的公园，带上简单的食物和饮料，享受户外时光。\n2. **动物园或植物园游览**：这些地方不仅能让孩子近距离接触各种动物和植物，还能增长知识。\n3. **博物馆参观**：许多城市都有丰富的博物馆资源，可以让孩子了解历史、艺术等多方面的知识。\n4. **亲子手工制作**：比如做手工艺品或者烘焙蛋糕，既能增进亲子关系，又能培养孩子的动手能力。\n5. **户外运动**：如骑自行车、徒步旅行或是简单的攀岩体验，既锻炼身体又能让孩子们接触大自然。\n\n希望这些建议能帮助到您！', 'generated_tokens_count': 156}
